# Probablistic Model

In [1]:
import math
import torch
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import preprocessing
from Training import model, utils, dataset, train
from sklearn.model_selection import train_test_split

import keras.backend as K

#### Load Data

In [2]:
data = utils.parseGainAndBWCsv2("Data/BW-3000.csv").astype(float)

In [3]:
print(data[0])

[6.200e+02 2.880e-06 3.775e+00 8.280e+10]


##### Split data into inputs X and target Y Normalize to [0,1] (inclusive) and create Training and Testign Splits


In [4]:
data = preprocessing.MinMaxScaler((0,1)).fit_transform(data)
X = data[:,:2]
Y = data[:,2:]
X_train, X_test, y_train, y_test = train_test_split(X, Y)

#### Define Model and Parameters

##### Define model

* Model: MLP with variable hidden layer number and width  
* Output layer: 
    * Node 1 and 2 are the mean for probability mean 
    * Node 3 and 4 are the diagonal of the covariance matrix


In [5]:
num_hidden_layers = 5
hidden_layer_size = 500

model = tf.keras.models.Sequential([])

for i in range(num_hidden_layers):
    model.add(tf.keras.layers.Dense(hidden_layer_size, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='sigmoid')) # add output layer: node 1,2 are 

2022-04-26 15:52:24.966911: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-26 15:52:24.966955: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-26 15:52:24.966984: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-reevest): /proc/driver/nvidia/version does not exist
2022-04-26 15:52:24.967275: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


##### Define Loss

* Loss: guassian_nll
    * cite: https://gist.github.com/sergeyprokudin/4a50bf9b75e0559c1fcd2cae860b879e

In [6]:
def gaussian_nll(ytrue, ypreds):
    """Keras implmementation of multivariate Gaussian negative loglikelihood loss function. 
    This implementation implies diagonal covariance matrix.
    
    Parameters
    ----------
    ytrue: tf.tensor of shape [n_samples, n_dims]
        ground truth values
    ypreds: tf.tensor of shape [n_samples, n_dims*2]
        predicted mu and logsigma values (e.g. by your neural network)
        
    Returns
    -------
    neg_log_likelihood: float
        negative loglikelihood averaged over samples
        
    This loss can then be used as a target loss for any keras model, e.g.:
        model.compile(loss=gaussian_nll, optimizer='Adam') 
    
    """
    print(f"ytrue: {ytrue.shape}")
    print(f"ypreds: {ypreds.shape}")
    n_dims = int(int(ypreds.shape[1])/2)
    mu = ypreds[:, 0:n_dims]
    logsigma = ypreds[:, n_dims:]
    sigma = ypreds[:, n_dims:]
    
    if K.any(sigma < 0):
        raise ValueError("sigma has negative entry/entries")
        
    #mse = -0.5*K.sum(K.square((ytrue-mu)/K.exp(logsigma)),axis=1)
    #sigma_trace = -K.sum(logsigma, axis=1)
    #log2pi = -0.5*n_dims*np.log(2*np.pi)
    
    
    log_likelihood = 0.5 * K.log(sigma**2) + (ytrue-mu)**2 / (sigma)

    return K.mean(log_likelihood)

In [7]:
loss_fn = gaussian_nll

optim = tf.keras.optimizers.Adam(
    learning_rate=0.001)
model.compile(optimizer=optim,
              loss=loss_fn)

In [8]:
model.fit(X_train, y_train, epochs=1000, batch_size= 700, validation_data=(X_test, y_test))

Epoch 1/1000


2022-04-26 15:52:25.068046: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


ytrue: (None, 2)
ypreds: (None, 4)


ValueError: in user code:

    /opt/conda/lib/python3.9/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /tmp/ipykernel_28166/3890421745.py:29 gaussian_nll  *
        raise ValueError("sigma has negative entry/entries")

    ValueError: sigma has negative entry/entries
